In [2]:
close all
clc
clear;

In [3]:

% definition of the variable

c=2.99792458e8;                               % unit is m/s
% fre=299.88e9;                                 % unit is Hz
fre=599.76e9;
d_object=0.05962;                               % unit is m
f=0.06;                                       % unit is m
step=0.0005;                                  % unit is m / step is equal to the pixel size
angle1=0/180*pi;                             % the tilt angle of the object(in degree) in lateral direction
angle2=-18/180*pi;                             % the tilt angle in othorganal of the object
K=0;                                          % PIXEL SHIFT FACTOR
O=0;                                          % shift option, shift the oddlines(c=0) or shift the even lines
AForXY=0;                                     % if AForXY=1 it is for phase and amplitute, if AForXY=0 it is X and Y chanel
turnornot=2;                                  % 1 is to inverse the figure 0 is not 2 is left and right inverse            
z=f;                                      % screen distance 
Elimit=3;
dynamic=-30;
Tick_range=20;
d_fix=0.004;
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

% load data

%%%%%%%%%%%%%%%%%%%%%%%%%%%

%%set the name of the object
objectname='Beam';    
%objectname='grid';
%objectname='starburst';


data_file = '10cm_5cm_focus_Siemensstar_10cm_guf_PHASE_Z96.50_ch1.dat';


%Euv0=load('/home/alex/ml_master/fourier_imaging_data_raw/Data/USA/1/16p66GHz_usa1part_10cm_2_Z104.00_ch1.dat');
%Auv0=load('/home/alex/ml_master/fourier_imaging_data_raw/Data/USA/1/16p66GHz_usa1part_10cm_2_Z104.00_ch2.dat');


%Euv0=load('/home/alex/ml_master/fourier_imaging_data_raw/Fourierdata/10cm_5cm_focus_Siemensstar_10cm_1_phase_L_Z96.50_ch1.dat');
%Auv0=load('/home/alex/ml_master/fourier_imaging_data_raw/Fourierdata/10cm_5cm_focus_Siemensstar_10cm_1_phase_L_Z96.50_ch2.dat');


Euv0=load('/home/alex/ml_master/virt/master_ml/cnn_fourier/Data_Augmentation/10cm_5cm_focus_1mm_phase_V_Z96.50_ch1.dat');
Auv0=load('/home/alex/ml_master/virt/master_ml/cnn_fourier/Data_Augmentation/10cm_5cm_focus_1mm_phase_V_Z96.50_ch2.dat');


[Hi Wi]=size(Euv0); % size of the data
disp(Hi);
disp(Wi);

[Euv1]=pixelshift(Euv0,K,O);
[Auv1]=pixelshift(Auv0,K,O);
[Euv]=ComplexData(Euv1,Auv1,AForXY);
% Euv=abs(Euv_d).*exp(i*angle(Euv));



[Euv1]=pixelshift(Euv0,K,O);
[Auv1]=pixelshift(Auv0,K,O);
[Euv_b]=ComplexData(Euv1,Auv1,AForXY);


% Euv_b=abs(Euv_b_d).*exp(i*Euv_b);

% %%%%%%%%%%%double line scan%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% [Hii Wii]=size(Euv_b);
% Euv_b=Euv_b(1:2:Hii,:);
% %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

[Hii Wii]=size(Euv_b);


if Hii<=Hi&&Wii<=Wi
    Euv_b=blkdiag(Euv_b,zeros(Hi-Hii,Wi-Wii));
elseif Hii>=Hi&&Wii>=Wi
    Euv_b=Euv_b(1:Hi,1:Wi);
elseif Hii>=Hi&&Wii<=Wi
    Euv_b=[Euv_b(1:Hi,:) zeros(Hi,Wi-Wii)];
else 
    Euv_b=[Euv_b(:,1:Wi) zeros(Hi-Hii,Wi)];
end
    


%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% This part is for X/Y chinal to Amplitute and phase
% Euv2=sqrt(Euv1.^2+Auv1.^2);
% Auv2=atan(Auv1./Euv1);
% 
% for m=1:Hi
%     for n=1:Wi
%         if Euv0(m,n)==0&&(Auv0(m,n)>0)
%             Auv2(m,n)=pi/2;
%         elseif Euv0(m,n)==0&&(Auv0(m,n)<0)
%             Auv2(m,n)=-pi/2;
%         elseif Euv0(m,n)==0&&(Auv0(m,n)==0)
%             Auv2(m,n)=0;
%         elseif (Euv0(m,n)<0)&&(Auv2(m,n)==0)
%             Auv2(m,n)=-pi;
%         elseif (Euv0(m,n)<0)&&(Auv2(m,n)<0)
%             Auv2(m,n)=Auv2(m,n)+pi;
%         elseif (Euv0(m,n)<0)&&(Auv2(m,n)>0)
%             Auv2(m,n)=Auv2(m,n)-pi;
%         end
%     end
% end
% Euv=Euv2.*exp(i*Auv2);                  % complex data

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%This part is for the data is just amplitute and phase
%Auv1=Auv1/0.9*pi;
%Euv=Euv1.*exp(i*Auv1);                  % complex data

[Euv_max Euv_rm Euv_vm]=maxloc(Euv);          % max value coordinate
u=1:Wi;
v=1:Hi;
u=step*(u-Euv_vm);                           % x axis coordinate of Fourier plane
v=step*(v-Euv_rm)';                           % y axis coordinate of Fourier plane
disp(u)
% Euv(102:Wi-2,:)=Euv(104:Wi,:);
% show data
Ieuv=10*log10(abs(Euv).^2/max(max(abs(Euv).^2)));
%figure(11)
imagesc(Ieuv)
% imagesc(Ieuv)
%imagesc(Euv0)
xlabel('x(mm)')
ylabel('y(mm)')
%title('Intensity USA')
%title('Intensity grid')
% title(strcat('Intensity-',objectname))
colorbar
axis equal, axis tight
set(gca,'linewidth',2,'fontsize',24,'fontname','Times');

[Exyout0,Ixyout0,x,y] = Inverse_fourier_function_tilt(fre,d_object,f,Euv,u,v,angle1,angle2);

figure(5)
imagesc(x*1000,y*1000,Ixyout0)
xlabel('x(mm)')
ylabel('y(mm)')
% %title('Intensity USA Ixyout')
% %title('Intensity grid Ixyout')
title(strcat('Intensity-',objectname,'Ixyout'))
axis equal, axis tight
colorbar
% creat file to store values of Ieuv

%fid = fopen('output.txt','w');
%fwrite(fid,Ieuv);
%fclose(fid);
Ieuv_inf = Ieuv;
Ieuv_inf(isinf(Ieuv_inf))=0;
writematrix(Ixyout0,'output_tab.dat', 'Delimiter', 'tab')
%save('output.txt','Ieuv', '-double','-tabs')

Euv_phase_temp=angle(Euv);
Euv_phase_temp(Euv_phase_temp<0)=Euv_phase_temp(Euv_phase_temp<0)+2*pi;

figure(12)
imagesc(u*1000,v*1000,Euv_phase_temp)
% imagesc(Euv_phase_temp)
xlabel('x(mm)')
ylabel('y(mm)')
%title('Phase USA')
%title('Phase grid')
% title(strcat('Phase-',objectname))
colorbar
axis equal, axis tight
set(gca,'linewidth',2,'fontsize',24,'fontname','Times');

[uu,vv]=meshgrid(u,v);


%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% figure
% mesh(uu*1000,vv*1000,abs(Euv))
% xlabel('(mm)')
% ylabel('(mm)')
% title('Intensity in Fourier plane')
% figure,mesh(uu*1000,vv*1000,angle(Euv))
% xlabel('(mm)')
% ylabel('(mm)')
% title('Phase in Fourier plane')
% % UPEuv=GoldsteinUnwrap2D_r1(angle(Euv));
% % figure,mesh(uu*1000,vv*1000,UPEuv)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

Ieuv_b=10*log10(abs(Euv_b).^2/max(max(abs(Euv_b).^2)));

figure(13)
imagesc(u*1000,v*1000,Ieuv_b)
%imagesc(Euv0)
xlabel('x(mm)')
ylabel('y(mm)')
%title('Intensity USA')
%title('Intensity grid')
 title(strcat('Intensity-',objectname,'-back'))
colorbar
axis equal, axis tight
set(gca,'linewidth',2,'fontsize',24,'fontname','Times');


Euv_phase_temp=angle(Euv_b);
Euv_phase_temp(Euv_phase_temp<0)=Euv_phase_temp(Euv_phase_temp<0)+2*pi;
figure(14)
imagesc(u*1000,v*1000,Euv_phase_temp)
xlabel('x(mm)')
ylabel('y(mm)')
%title('Phase USA')
%title('Phase grid')
title(strcat('Phase-',objectname,'-back'))
colorbar
axis equal, axis tight
set(gca,'linewidth',2,'fontsize',24,'fontname','Times');


% Make the maximum value as the center of the image
%Euv_size=size(Euv)
[Euv_new Euv_new_b Z]=expandM_back(Euv,Euv_b);
%[X Y]=expandM_back(M,N)
%ESIZE1=size(Euv_new)
% Euv_new0=minifyM(Euv);
disp(size(Euv_new_b))

[Hi_new Wi_new]=size(Euv_new);
[Euv_new_max Euv_new_rm Euv_new_vm]=maxloc(Euv_new);


% change the center by hand
%Euv_new_rm=Euv_new_rm;
%Euv_new_vm=Euv_new_vm;

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%lowpass filter
%Efactor=zeros(Hi_new,Wi_new);
%Efactor(62:Hi_new,61:Wi_new)=ones(Hi_new-62+1,Wi_new-61+1);
%Euv_new=Euv_new.*Efactor;
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
u=1:Wi_new;
v=1:Hi_new;
u=step*(u-Euv_new_vm);
%AA=u;
v=step*(v-Euv_new_rm)';
%BB=v;

% % show the centered picture
% figure(3)
% imagesc(u*1000,v*1000,abs(Euv_new))
% xlabel('x(mm)')
% ylabel('y(mm)')
% %title('Intensity USA')
% %title('Intensity grid center')
% title(strcat('Intensity-',objectname,'after centered'))
% colorbar
% axis equal, axis tight
% %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% figure(31)
% imagesc(u*1000,v*1000,angle(Euv_new))
% xlabel('x(mm)')
% ylabel('y(mm)')
% %title('Intensity USA')
% %title('Intensity grid center')
% title(strcat('Intensity-',objectname,'after centered'))
% colorbar


% axis equal, axis tight
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%size(u)   % for debug 
%size(v)   % for debug
[Exyout0,Ixyout0,x,y] = Inverse_fourier_function_tilt(fre,d_object,f,Euv_new,u,v,angle1,angle2);

figure(4)
imagesc(x*1000,y*1000,Ixyout0)
xlabel('x(mm)')
ylabel('y(mm)')
% %title('Intensity USA Ixyout')
% %title('Intensity grid Ixyout')
title(strcat('Intensity-',objectname,'Ixyout'))
axis equal, axis tight
colorbar
% axis([-50 50 -50 50])

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

% This part is to enlarge the picture

Euv_new=enlargeM(enlargeM(Euv_new));
Euv_new_b=enlargeM(enlargeM(Euv_new_b));

disp(size(Euv_new))

[Hi_new Wi_new]=size(Euv_new); 
% figure(51)
% imagesc(abs(Euv_new))

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% % If MASKED THE DATA
% masksize=42/1000;                                            % size of the pass plane 
% Euv_new_=zeros(Hi_new,Wi_new);
% for i=1:Hi_new
%     for j=1:Wi_new
%         if abs(i-(Hi_new+1)/2)<=(masksize/step)&&abs(j-(Wi_new+1)/2)<=(masksize/step)
%             Euv_new_(i,j)=1;
%         end
%     end
% end
% Euv_new=Euv_new_.*Euv_new;
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% figure(52)
% imagesc(abs(Euv_new))

[Euv_new_max Euv_new_rm Euv_new_vm]=maxloc(Euv_new);
%size(Euv_new)
u=1:Wi_new;
v=1:Hi_new;
u=step*(u-Euv_new_vm);
v=step*(v-Euv_new_rm)';

% show the picture
% figure(5)
% imagesc(u*1000,v*1000,abs(Euv_new))
% xlabel('x(mm)')
% ylabel('y(mm)')
% %title('Intensity USA after expand')
% %title('Intensity grid after expand')
% title(strcat('Intensity-',objectname,' after expand'))
% colorbar
% axis equal, axis tight

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%  d_object2=ones(1,5);
%   for m=1:5
%       d_object2(m)=d_object+(m-3)*0.005;  
%   end
d_object2=d_object;
% d_object2=d_object-0.02:0.01:d_object+0.02;
% d_object2=0.021:0.02:0.101;
% d_object2=0.02:0.0001:0.10;
% d_object2=d_object-0.01:0.001:d_object+0.01;
% d_object2=[ 0.115 0.18];  
%d_object2=[0.08 0.085 0.09 0.095 0.10 0.105 0.11 0.115 0.12 0.125]; 
%figure
back_or_not=0;
[Eintensity Epashe x y]=multioutputforprofileback(fre,d_object2,f,Euv_new,u,v,angle1,angle2,objectname,turnornot,Tick_range,dynamic,back_or_not);
back_or_not=0;
[Eintensity_b Epashe_b x y]=multioutputforprofileback(fre,d_object2+d_fix,f,Euv_new_b,u,v,0,0,objectname,turnornot,Tick_range,dynamic,back_or_not);
% close all
[Hi Wi Li]=size(Eintensity);
for m=1:Li
    
%     E_temp_=Eintensity_b(:,:,m);
%     E_temp_(abs(E_temp_)<=1e-9)=1e-9*exp(i*angle( E_temp_(abs(E_temp_)<=3e-18)));
%     Eintensity_b(:,:,m)=E_temp_;
    Eintensity(:,:,m)=Eintensity(:,:,m)./Eintensity_b(:,:,m);
    Epashe(:,:,m)=Epashe(:,:,m)-Epashe_b(:,:,m);
 
% % figure
% % Intensity_temp=abs(Eintensity(:,:,m));
% % Intensity_temp(Intensity_temp>2)=2;
% % Intensity_temp=(Intensity_temp).^2./max(max((Intensity_temp).^2));
% % Intensity_temp=60*Intensity_temp-60;
% % imagesc(x*1000,y*1000,Intensity_temp)
% % %mesh(xx*1000,yy*1000,Ixyout)
% % xlabel('x(mm)','linewidth',2,'fontsize',18,'fontname','Times') 
% % ylabel('y(mm)','linewidth',2,'fontsize',18,'fontname','Times')
% % 
% % % %title(strcat('Intensity-',objectname1))
% % load Mymap;
% % colormap(Mymap);
% % colorbar;
% % axis equal, axis tight
% % axis([-35 35 -35 35])
% % set(gca,'linewidth',2,'fontsize',18,'fontname','Times');
% % set(gca,'XTick',[-30:10:30])
E_temp=Eintensity(:,:,m);
E_temp(E_temp>Elimit)=Elimit;
Eintensity(:,:,m)=E_temp;

% %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
 
ILOG=10*log10((abs(Eintensity(:,:,m))).^2/(max(max(abs(Eintensity(:,:,m)))))^2);
ILOG(ILOG<dynamic)=dynamic;
figure
E_temp=abs(Eintensity(:,:,m)).^2;
E_temp(E_temp>20)=0;
imagesc(x*1000,y*1000,E_temp);
%imagesc(x*1000,y*1000,ILOG')
%imwrite(E_temp, sprintf('%d.png',m))
%dlmwrite('output.dat', E_temp, 'delimiter', ',');  %%%%%%% write data file
%of one of the images

% xlabel('x(mm)','linewidth',2,'fontsize',18,'fontname','Times') 
% ylabel('y(mm)','linewidth',2,'fontsize',18,'fontname','Times')
%title(strcat('Intensity-log-',objectname1))
% load Mymap;
% colormap(Mymap);
colorbar;

xlabel('x(mm)')
ylabel('y(mm)')

axis equal, axis tight
axis([-Tick_range Tick_range -Tick_range Tick_range])
set(gca,'linewidth',2,'fontsize',24,'fontname','Times');
set(gca,'XTick',[-Tick_range:10:Tick_range])
% axis off
% % % 
% % %
% if m>2
phase_temp= Epashe(:,:,m);
phase_temp(phase_temp>4)=phase_temp(phase_temp>4)-2*pi;
phase_temp(phase_temp<-4)=phase_temp(phase_temp<-4)+2*pi;
% phase_unwrap_temp=phase_temp;
% phase_unwrap_temp(140:190,140:190)=GoldsteinUnwrap2D_r1(phase_temp(140:190,140:190));
% % 
figure
%mesh(xx*1000,yy*1000,phase_temp)
imagesc(x*1000,y*1000,phase_temp')
xlabel('x(mm)','linewidth',2,'fontsize',24,'fontname','Times') 
ylabel('y(mm)','linewidth',2,'fontsize',24,'fontname','Times')
%title(strcat('Recover-Phase-',objectname1,'beforeunwrap'))
% load Mymap;
% colormap(Mymap);
colorbar;

%axis equal, axis tight
axis([-Tick_range Tick_range -Tick_range Tick_range])
set(gca,'linewidth',2,'fontsize',24,'fontname','Times');
set(gca,'XTick',[-Tick_range:10:Tick_range])
end
    


   120

   120



'pixelshift' is not found in the current folder or on the MATLAB path, but exists in:
    /home/alex/ml_master/fourier_imaging_data_raw/Fourier_reconstruction

Change the MATLAB current folder or add its folder to the MATLAB path.

